Exploratory analysis of the review data to find:
* total numbers and general descriptions of non-empty reviews in the database.
* total number of reviews per year in the UK.
* distribution (with error bars) of the reviews per month since 2006.

In [1]:
import pyodbc
dwh_login = {'option': '3',
             'database': 'datawarehouse',
             'driver': '{PostgreSQL ANSI}',
             'server': 'je-dw.cspauv1pfeh3.eu-west-1.redshift.amazonaws.com',
             'pwd': '',
             'user': '',
             'port': '5439'}

sql = 'select 1;'

try:
    schema_conn = pyodbc.connect('DSN=Redshift x64')
    schema_db = schema_conn.cursor()
    data = schema_db.execute(sql).fetchall()
    schema_conn.commit()
#    schema_db.close()
#    schema_conn.close()
except Exception, ex:
    raise

print data

[(1, )]


In [2]:
import pandas as pd
q1="""
select * from ecom.rating
limit 10
"""
#count the rows in a table (count=257252380 for ecom.rating)
q2="""
select count(*) from ecom.rating
"""
#get non-emty rows in a table 
q3="""
select commentary from ecom.rating where commentary<>'' and country_key=8
limit 1000
"""
#count non-empty rows in the rating table
q4="""
select count(*) from ecom.rating where commentary<>'' and country_key=8
"""
#count non-empty rows in the rating table
q5="""
select count(*) from ecom.rating where commentary<>''
"""


dfA = pd.read_sql(q2, schema_conn)
dfB = pd.read_sql(q4, schema_conn)
dfC = pd.read_sql(q5, schema_conn)

#ss=df['commentary'][:6]
#print ss
print "the total number of UK reviews is:", dfB
print "% rows with UK reviews in the rating table:", (dfB/dfA)*100
print "% UK reviews considering the total number of existing reviews:", (dfB/dfC)*100 

the total number of UK reviews is:      count
0  4917650
% rows with UK reviews in the rating table:       count
0  1.834272
% UK reviews considering the total number of existing reviews:        count
0  77.323012


In [11]:
#What is the number of customer reviews per year in the ecom.rating table
#%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
q1="""
select count(*) from ecom.rating where commentary<>'' and country_key=8 and datepart(year,rate_date)=2006
"""
q2="""
select datepart(year,rate_date),count(*) from ecom.rating where commentary<>'' and country_key=8
group by datepart(year,rate_date)
order by datepart(year,rate_date);
"""
df = pd.read_sql(q2, schema_conn)
plt.figure(); 
df.plot.bar(x="date_part",y="count",legend=False,color='r')
plt.xlim([0,9.5])
plt.xlabel("", fontsize=25)
plt.ylabel("# of reviews", fontsize=25)
plt.tick_params(axis='both', which='major', labelsize=20)
plt.xticks(rotation=45)
plt.show()


In [4]:
#What is the total distribution of reviews per month.
q="""
select datepart(month,rate_date),count(*) from ecom.rating where commentary<>'' and country_key=8
group by datepart(month,rate_date)
order by datepart(month,rate_date);
"""
df = pd.read_sql(q, schema_conn)
plt.figure(); 
df.plot.bar(x="date_part",y="count",legend=False)
plt.title("From 2006 to 2016")
plt.xlabel("Month")
plt.ylabel("Total # of reviews")
plt.show()

In [5]:
# what is the month variability of the percentage of th reviews from 2007 to 2015? 
# In this analysis we center our calculation on the years that are 'complete', that is, that have written reviews
# in every month. That is why we explude the year 2006 and the current year 2016
years=(2007,2008,2009,2010,2011,2012,2013,2014,2015)
#quit fix functions in case we decide to use the years 2006 and 2016 (which are incomplete)
#function to fix 2006 data (including '0' in the missing values of the 2006 table)
def fix2006(df):
    df.loc[8]=[1,0]
    df.loc[9]=[2,0]
    df.loc[10]=[3,0]
    df.loc[11]=[5,0]
    df=df.sort_values(by='date_part', ascending=True)
    return df
#function to fix 2016 data (including '0' in the missing values of the 2016 table)
def fix2016(df):
    df.loc[1]=[2,0]
    df.loc[2]=[3,0]
    df.loc[3]=[4,0]
    df.loc[4]=[5,0]
    df.loc[5]=[6,0]
    df.loc[6]=[7,0]
    df.loc[7]=[8,0]
    df.loc[8]=[9,0]
    df.loc[9]=[10,0]
    df.loc[10]=[11,0]
    df.loc[11]=[12,0]
    df=df.sort_values(by='date_part', ascending=True)
    return df
###########################################################################
#creating a data frame that will be calculated like this:
#(1) for a year we obtain the number of reviews on each month
#(2) we express these number as a percentage of the total number of reviews in that year
#(3) the 12 percentage values will then form a column on a bigger table that will contain the years 2007-2015
#(4) we then calculate the variations of these percentages, 
#    that is for we will see the mean and std of the percentage for 'january' from 2007 to 2015... for 'February'.., etc.
#########################################################################
df = pd.DataFrame(index=range(0,12),columns=['first'], dtype='float')
for y in years:
    q="""
select datepart(month,rate_date),count(*) from ecom.rating where commentary<>'' and country_key=8 and datepart(year,rate_date)={}
group by datepart(month,rate_date)
order by datepart(month,rate_date);
""".format(str(y))
    df1 = pd.read_sql(q, schema_conn)
    df1["count"]=(df1["count"]/df1.sum()[1])*100
    df=pd.concat([df, df1["count"]], axis=1)
df=df.drop(df.columns[[0]], axis=1)

###  plotting ###
N=12
meanM=list(df.mean(axis=1))
stdM=list(df.std(axis=1))
xMlabels=(('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'))
fig, ax = plt.subplots()
ax.set_ylabel('% of reviews', fontsize=25)
ax.set_title('From 2007 to 2015',fontsize=35)
ax.set_xlabel('Months',fontsize=25)
ind = np.arange(N)
width = 0.80
plt.bar(ind, meanM, width, color='r', yerr=stdM)
plt.xticks(ind + width/2., xMlabels)
plt.tick_params(axis='both', which='major', labelsize=20)
plt.xticks(rotation=30)
plt.show()  

In [6]:
#what is the distribution of reviews per month in a specific year?
q2006="""
select datepart(month,rate_date),count(*) from ecom.rating where commentary<>'' and country_key=8 and datepart(year,rate_date)=2006
group by datepart(month,rate_date)
order by datepart(month,rate_date);
"""
q2007="""
select datepart(month,rate_date),count(*) from ecom.rating where commentary<>'' and country_key=8 and datepart(year,rate_date)=2007
group by datepart(month,rate_date)
order by datepart(month,rate_date);
"""
q2008="""
select datepart(month,rate_date),count(*) from ecom.rating where commentary<>'' and country_key=8 and datepart(year,rate_date)=2008
group by datepart(month,rate_date)
order by datepart(month,rate_date);
"""
q2009="""
select datepart(month,rate_date),count(*) from ecom.rating where commentary<>'' and country_key=8 and datepart(year,rate_date)=2009
group by datepart(month,rate_date)
order by datepart(month,rate_date);
"""
q2010="""
select datepart(month,rate_date),count(*) from ecom.rating where commentary<>'' and country_key=8 and datepart(year,rate_date)=2010
group by datepart(month,rate_date)
order by datepart(month,rate_date);
"""
q2011="""
select datepart(month,rate_date),count(*) from ecom.rating where commentary<>'' and country_key=8 and datepart(year,rate_date)=2011
group by datepart(month,rate_date)
order by datepart(month,rate_date);
"""
q2012="""
select datepart(month,rate_date),count(*) from ecom.rating where commentary<>'' and country_key=8 and datepart(year,rate_date)=2012
group by datepart(month,rate_date)
order by datepart(month,rate_date);
"""
q2013="""
select datepart(month,rate_date),count(*) from ecom.rating where commentary<>'' and country_key=8 and datepart(year,rate_date)=2013
group by datepart(month,rate_date)
order by datepart(month,rate_date);
"""
q2014="""
select datepart(month,rate_date),count(*) from ecom.rating where commentary<>'' and country_key=8 and datepart(year,rate_date)=2014
group by datepart(month,rate_date)
order by datepart(month,rate_date);
"""
q2015="""
select datepart(month,rate_date),count(*) from ecom.rating where commentary<>'' and country_key=8 and datepart(year,rate_date)=2015
group by datepart(month,rate_date)
order by datepart(month,rate_date);
"""
q2016="""
select datepart(month,rate_date),count(*) from ecom.rating where commentary<>'' and country_key=8 and datepart(year,rate_date)=2016
group by datepart(month,rate_date)
order by datepart(month,rate_date);
"""
########################################
df2006 = pd.read_sql(q2006, schema_conn)
df2007 = pd.read_sql(q2007, schema_conn)
df2008 = pd.read_sql(q2008, schema_conn)
df2009 = pd.read_sql(q2009, schema_conn)
df2010 = pd.read_sql(q2010, schema_conn)
df2011 = pd.read_sql(q2011, schema_conn)
df2012 = pd.read_sql(q2012, schema_conn)
df2013 = pd.read_sql(q2013, schema_conn)
df2014 = pd.read_sql(q2014, schema_conn)
df2015 = pd.read_sql(q2015, schema_conn)
df2016 = pd.read_sql(q2016, schema_conn)
########################################
plt.figure(); 
df2006.plot.bar(x="date_part",y="count",legend=False)
plt.title("2006")
plt.xlabel("Month")
plt.ylabel("Number of reviews")
plt.show()
##
plt.figure(); 
df2007.plot.bar(x="date_part",y="count",legend=False)
plt.title("2007")
plt.xlabel("Month")
plt.ylabel("Number of reviews")
plt.show()
##
plt.figure(); 
df2008.plot.bar(x="date_part",y="count",legend=False)
plt.title("2008")
plt.xlabel("Month")
plt.ylabel("Number of reviews")
plt.show()
##
plt.figure(); 
df2009.plot.bar(x="date_part",y="count",legend=False)
plt.title("2009")
plt.xlabel("Month")
plt.ylabel("Number of reviews")
plt.show()
##
plt.figure(); 
df2010.plot.bar(x="date_part",y="count",legend=False)
plt.title("2010")
plt.xlabel("Month")
plt.ylabel("Number of reviews")
plt.show()
##
plt.figure(); 
df2011.plot.bar(x="date_part",y="count",legend=False)
plt.title("2011")
plt.xlabel("Month")
plt.ylabel("Number of reviews")
plt.show()
##
plt.figure(); 
df2012.plot.bar(x="date_part",y="count",legend=False)
plt.title("2012")
plt.xlabel("Month")
plt.ylabel("Number of reviews")
plt.show()
##
plt.figure(); 
df2013.plot.bar(x="date_part",y="count",legend=False)
plt.title("2013")
plt.xlabel("Month")
plt.ylabel("Number of reviews")
plt.show()
##
plt.figure(); 
df2014.plot.bar(x="date_part",y="count",legend=False)
plt.title("2014")
plt.xlabel("Month")
plt.ylabel("Number of reviews")
plt.show()
##
plt.figure(); 
df2015.plot.bar(x="date_part",y="count",legend=False)
plt.title("2015")
plt.xlabel("Month")
plt.ylabel("Number of reviews")
plt.show()
##
plt.figure(); 
df2016.plot.bar(x="date_part",y="count",legend=False)
plt.title("2016")
plt.xlabel("Month")
plt.ylabel("Number of reviews")
plt.show()
